### 1. Importing the libraries.

In [2]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import os

### 2. Setting Twitter Credentials.

In [3]:
consumer_key = 'consumer_key'
consumer_secret = 'consumer_secret'
access_key= 'access_key'
access_secret = 'access_secret'

### 3. Passing the Twitter credentials to tweepy.

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

### 4. Define columns of the csv file.

In [5]:
COLS = ['id', 'created_at', 'source', 'original_text', 'lang',
'favorite_count', 'retweet_count', 'original_author', 'possibly_sensitive', 'hashtags',
'user_mentions', 'place', 'place_coord_boundaries']

### 5. Define a function write_tweets for extracting the tweets.

In [6]:
def write_tweets(keyword, since, until, file):
    #If the file exists, then read the existing data from the CSV file.
    if os.path.exists(file):
        df = pd.read_csv(file, header=0)
    else:
        df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    page = tweepy.Cursor(api.search, q=keyword, count=200,since = since, until = until, include_rts=False).items()
    for status in page:
        new_entry = []
        status = status._json
        new_entry += [status['id'], status['created_at'], status['source'], status['text'], status['lang'], status['favorite_count'], status['retweet_count']]
        new_entry.append(status['user']['screen_name'])
        try:
            is_sensitive = status['possibly_sensitive']
        except KeyError:
            is_sensitive = None
        new_entry.append(is_sensitive)
        hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
        new_entry.append(hashtags) #append the hashtags
        mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
        new_entry.append(mentions) #append the user mentions

        try:
            coordinates = [coord for loc in status['place']   ['bounding_box']['coordinates'] for coord in loc]
        except TypeError:
            coordinates = None
        new_entry.append(coordinates)

        try:
            location = status['user']['location']
        except TypeError:
            location = None
        new_entry.append(location)
        single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
        df = df.append(single_tweet_df, ignore_index=True)
 
    csvFile = open(file, 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

### 6. Setting arguments for extacting tweets by write_tweet function.
movie_keywords = Name of the movie.<br>
date_since = Date since which tweets are to be extracted.<br>
date_until = Date till which tweets are to be extracted.<br>
movie_tweets = Name/Address of Output file.<br>

In [9]:
movie_keywords = '#angreziMedium'
date_since = "2020-03-12"
date_until = "2020-03-13"
movie_tweets = "angreziMedium.csv"

### 7. Calling the function and Extracting tweets.

This will give us a csv file containing tweets and related data.<br>
This might take some time to execute.

In [ ]:
write_tweets(movie_keywords, date_since, date_until, movie_tweets)